# Cross Validation for IDW Interpolation 
## Task 2A (continuous)

This document includes Python codes that conduct cross validation (CV) for Inverse Distance Weighting (IDW) Interpolation on water quality parameters, including 4 water quality parameters in arcpy environment:
- Dissolved oxygen (DO_mgl)
- Salinity (Sal_ppt)
- Turbidity (Turb_ntu)
- Temperature (T_c)

The analysis is conducted in the separate water bodies:
- Guana Tolomato Matanzas (GTM)
- Estero Bay (EB)
- Charlotte Harbor (CH)
- Biscayne Bay (BB)
- Big Bend Seagrasses (BBS)

**Tasks:**  

- Task 2A Calculate the RMSE and Mean Error (ME) for IDW results using both continuous and discrete data

- **Task 2B Calculate the RMSE and Mean Error (ME) for IDW results using continuous data.**


**Monthly: The SEACAR team would like the USF team to define their months as six 30-day
increments prior to the storm day, and then six 30-day increments following the storm day.** 

**Weekly: The SEACAR team would like the USF team to define their weeks as 26 7-day
increments prior to the storm day, and then 26 7-day increments following the storm day** 

**Contents:**
* [1. Data Preprocess](#reg_preprocessing)
    * [1.1 Subsetting Dataset](#reg_subset)
    * [1.2 Preview Dataset](#reg_preview)
    * [1.3 Fill Unique ID](#reg_id)
* [2. Create Shapefile](#reg_create_shp)
* [3. Cross Validation for IDW](#reg_cv_idw)
* [4. Monthly Results](#month)

In [1]:
import pandas as pd
import numpy as np
import arcpy
from arcpy.sa import *
import os
import math
import warnings

import importlib
import sys
path = r'F:\SEACAR_WQ_2024\git\misc'
# path = r'E:\Projects\SEACAR_WQ_2024\git\misc'

sys.path.insert(0, path)
# import idw_rk
import idw_rk_x
# importlib.reload(idw_rk)
importlib.reload(idw_rk_x)

import pyproj

# define scratch folder to avoid overwritting from parallel threats

arcpy.env.scratchWorkspace = r"F:\SEACAR_WQ_2024\scratch/IDW_con"

warnings.filterwarnings('ignore')

## 1. Data Preprocessing <a class="anchor" id="reg_preprocessing"></a>
### 1.1 Load csv files

In [2]:
gis_path = r'F:\SEACAR_WQ_2024/GIS_Data/'
# gis_path = r'E:\Projects\SEACAR_WQ_2024/GIS_Data/'
dfCon = pd.read_csv(gis_path + 'OEAT_Continuous_WQ-2024-Feb-21.csv', low_memory=False)

### Define output folder. Here is the folder setting for week.

In [3]:
# shpCon_folder = gis_path + r"shapefiles_Con/week"
# idwCon_folder = gis_path + r"idw_Con/week"

shpCon_folder = gis_path + r"shapefiles/idw_week"
idwCon_folder = gis_path + r"raster_output/idw_week"

### 1.2 Subsetting Data <a class="anchor" id="reg_subset"></a>

#### Include the time period from 9 am to 17 pm in a day

In [4]:
dfCon['SampleDate'] = pd.to_datetime(dfCon['SampleDate'], format='%Y-%m-%d %H:%M:%S.%f')

# Include date from 9:00 am to 17:00 pm
start_time = '09:00'
end_time = '17:00'

dfConTime = dfCon[dfCon['SampleDate'].dt.time.between(pd.to_datetime(start_time).time(), pd.to_datetime(end_time).time())]
dfConTime["SampleDate"] = pd.to_datetime(dfConTime['SampleDate'])
dfConTime.head()

,Unnamed: 0,RowID,ProgramID,ParameterName,ParameterUnits,ProgramLocationID,ActivityType,SampleDate,Year,Month,RelativeDepth,ResultValue,Latitude_DD,Longitude_DD,ManagedAreaName,AreaID,SEACAR_QAQCFlagCode,WaterBody,WbodyAcronym,Season
0,0,1,512,Water Temperature,Degrees C,CHWW1,NaN,2021-11-25 11:45:00,2021,11,bottom,19.2,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
1,1,2,512,Water Temperature,Degrees C,CHWW1,NaN,2021-11-13 16:15:00,2021,11,bottom,22.3,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
2,2,3,512,Water Temperature,Degrees C,CHWW1,NaN,2021-10-26 14:15:00,2021,10,bottom,28.3,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
3,3,4,512,Water Temperature,Degrees C,CHWW1,NaN,2022-09-16 12:45:00,2022,9,bottom,29.7,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall
4,4,5,512,Water Temperature,Degrees C,CHWW1,NaN,2022-09-20 10:30:00,2022,9,bottom,29.3,26.8325,-82.14805,Cape Haze Aquatic Preserve,9,6Q,Charlotte Harbor,CH,Fall


### 1.3 Select the data based on the 52 weeks excel file.

In [6]:
area_ab = ["GTM","EB","CH","BB","BBS"]
period_type = [" 52 week"," Month"]
dfConTime["Period"] = ""
def select_data_period(df,area,period):
    sheet_name = str(area) + str(period)
    df_period_table = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name)
    df_select_area = df[df["WbodyAcronym"]==str(area)]
    df_period_table['Start Date'] = pd.to_datetime(df_period_table['Start Date'])
    df_period_table['End Date']   = pd.to_datetime(df_period_table['End Date'])
    sub_dfs = []

    for index, row in df_period_table.iterrows():
        start_date = row['Start Date']
        end_date   = row['End Date']
        sub_df = df_select_area[(df_select_area['SampleDate'] >= start_date) & (df_select_area['SampleDate'] < end_date)]
        sub_df['Period'] = row["Week"]
        sub_dfs.append(sub_df)
        
    df_period = pd.concat(sub_dfs,ignore_index=True)
    return df_period

### 1.4 Generate the aggregated mean value for weekly data.

In [7]:
sel_week_temp = []
for each in area_ab:
    df_week_temp = select_data_period(dfConTime,str(each)," 52 week")
    df_week_temp_group = df_week_temp.groupby(['WaterBody','ParameterName','ParameterUnits',
                                          'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym',"Period"])["ResultValue"].agg("mean").reset_index()
    sel_week_temp.append(df_week_temp_group)
df_week_select_Mean = pd.concat(sel_week_temp,ignore_index=True)

In [9]:
df_week_select_Mean.head(50)

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,Period,ResultValue
0,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,23,5.566667
1,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,24,5.106926
2,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,25,4.686580
3,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,26,5.511688
4,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,27,2.891775
5,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,28,3.744156
6,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,29,3.986147
7,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,30,4.797541
8,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,32,5.752941
9,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,33,6.962771


### 1.5 Convert coordinate system to EPSG: 3086<a class="anchor" id="reg_coordinate"></a>


In [11]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
df_week_select_Mean[['x', 'y']] = df_week_select_Mean.apply(transform_coordinates, axis=1)

## 2. Prepare for batch interpolation<a class="anchor" id="reg_batch"></a>

In [38]:
area_shortnames = {
    'Guana Tolomato Matanzas': 'GTM',
    'Estero Bay': 'EB',
    'Charlotte Harbor': 'CH',
    'Biscayne Bay': 'BB',
    'Big Bend Seagrasses':'BBS'
}

param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

### 2.2 Define the barrier files

In [39]:
barrier_folder = os.path.join(gis_path, 'Barriers')
barrier_folder

barriers = []
for file in os.listdir(barrier_folder):
    if file.endswith(".shp"):
        barriers.append(os.path.join(barrier_folder, file))

for barrier in barriers:
    print(barrier)

F:\SEACAR_WQ_2024/GIS_Data/Barriers\BBS_Barriers.shp
F:\SEACAR_WQ_2024/GIS_Data/Barriers\BB_Barriers.shp
F:\SEACAR_WQ_2024/GIS_Data/Barriers\CH_Barriers.shp
F:\SEACAR_WQ_2024/GIS_Data/Barriers\EB_Barriers.shp
F:\SEACAR_WQ_2024/GIS_Data/Barriers\GTM_Barriers.shp


### 2.3 Define waterbody boundary for spatial extent and masking

In [40]:
waterbody_extent = os.path.join(gis_path, 'OEAT_Waterbody_Boundaries', 'OEAT_Waterbody_Boundary.shp')

unique_waterbodies = []
with arcpy.da.SearchCursor(waterbody_extent, ['WaterbodyA']) as cursor:
    for row in cursor:
        unique_waterbodies.append(row[0])

print("Unique Waterbodies:", unique_waterbodies)

Unique Waterbodies: ['BBS', 'BB', 'CH', 'EB', 'GTM']


### 2.4 Define and generate weekly table.

In [16]:
waterBody = ['Big Bend Seagrasses', 'Biscayne Bay', 'Charlotte Harbor', 'Estero Bay', 'Guana Tolomato Matanzas']
parameter = ['Dissolved Oxygen', 'Salinity', 'Turbidity', 'Water Temperature']
waterBody_list = []
parameter_list= []
week_list = []
for i in waterBody:
    for j in parameter:
        for k in range(1,53):
            waterBody_list.append(i)
            parameter_list.append(j)
            week_list.append(k)

In [17]:
df_week_table = pd.DataFrame({
    "WaterBody":waterBody_list,
    "Parameter":parameter_list,
    "Period":week_list
})
df_week_table["Filename"] = ""
df_week_table["NumDataPoints"] = ""
df_week_table["RMSE"] = ""
df_week_table["ME"] = ""
df_week_table["WbodyAcronym"] = df_week_table["WaterBody"].map(area_shortnames)

In [18]:
name_ab = df_week_table["WbodyAcronym"].unique()
para_list = df_week_table["Parameter"].unique()
dfs_sub = []
for each in name_ab:
    sheet_name_use = str(each) + " 52 week"
    df_period_table_used = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name_use)
    startDate = list(df_period_table_used["Start Date"])
    endDate   = list(df_period_table_used["End Date"])
    for para in para_list:
        df_temp_use = df_week_table[(df_week_table["WbodyAcronym"]==str(each))&(df_week_table["Parameter"]==str(para))]
        df_temp_use["startDate"] = startDate
        df_temp_use["endDate"]   = endDate
        dfs_sub.append(df_temp_use)
df_week_table = pd.concat(dfs_sub)
df_week_table["Year"] = df_week_table["startDate"].dt.year
df_week_table.to_csv(gis_path + 'week_table.csv')

# 1.2 Preview Dataset <a class="anchor" id="reg_preview"></a>

In [20]:
df_week_select_Mean.head(50)

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,Period,ResultValue,x,y
0,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,23,5.566667,665054.340859,631868.366218
1,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,24,5.106926,665054.340859,631868.366218
2,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,25,4.686580,665054.340859,631868.366218
3,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,26,5.511688,665054.340859,631868.366218
4,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,27,2.891775,665054.340859,631868.366218
5,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,28,3.744156,665054.340859,631868.366218
6,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,29,3.986147,665054.340859,631868.366218
7,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,30,4.797541,665054.340859,631868.366218
8,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,32,5.752941,665054.340859,631868.366218
9,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,33,6.962771,665054.340859,631868.366218


## 1.3 Fill NaN RowID with unique ID, IDW function needs unique ID <a class="anchor" id="reg_id"></a>

In [21]:
idw_rk_x.fill_nan_rowids(df_week_select_Mean, 'RowID')

df_week_select_Mean['RowID'] = df_week_select_Mean['RowID'].astype(int)
df_week_select_Mean

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,Period,ResultValue,x,y,RowID
0,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,23,5.566667,665054.340859,631868.366218,1
1,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,24,5.106926,665054.340859,631868.366218,2
2,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,25,4.686580,665054.340859,631868.366218,3
3,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,26,5.511688,665054.340859,631868.366218,4
4,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,27,2.891775,665054.340859,631868.366218,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,49,20.032828,455952.242067,626846.365120,3453
3453,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,50,20.496970,455952.242067,626846.365120,3454
3454,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,51,16.403535,455952.242067,626846.365120,3455
3455,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,52,21.236364,455952.242067,626846.365120,3456


# 2. Create Shapefile <a class="anchor" id="reg_create_shp"></a>

In [17]:
idw_rk_x.delete_all_files(shpCon_folder)

In [18]:
# Merge interested with latitude and longitude columns
seasons_con_coord = idw_rk_x.merge_with_lat_long1(df_week_table, df_week_select_Mean)

In [19]:
seasons_con_coord

,WaterBody,Parameter,Period,Filename,NumDataPoints,RMSE,ME,WbodyAcronym,startDate,endDate,Year,x,y,RowID,ResultValue
0,Big Bend Seagrasses,Dissolved Oxygen,1,,,,,BBS,2021-01-06,2021-01-12,2021,455952.242067,626846.365120,3286,8.345960
1,Big Bend Seagrasses,Dissolved Oxygen,2,,,,,BBS,2021-01-13,2021-01-19,2021,455952.242067,626846.365120,3287,8.507576
2,Big Bend Seagrasses,Dissolved Oxygen,3,,,,,BBS,2021-01-20,2021-01-26,2021,455952.242067,626846.365120,3288,8.120202
3,Big Bend Seagrasses,Dissolved Oxygen,4,,,,,BBS,2021-01-27,2021-02-02,2021,455952.242067,626846.365120,3289,7.933838
4,Big Bend Seagrasses,Dissolved Oxygen,5,,,,,BBS,2021-02-03,2021-02-09,2021,455952.242067,626846.365120,3290,8.726263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3491,Guana Tolomato Matanzas,Water Temperature,51,,,,,GTM,2017-03-25,2017-04-01,2017,653509.737698,674232.732990,747,22.775325
3492,Guana Tolomato Matanzas,Water Temperature,52,,,,,GTM,2017-04-01,2017-04-08,2017,665054.340859,631868.366218,733,24.606061
3493,Guana Tolomato Matanzas,Water Temperature,52,,,,,GTM,2017-04-01,2017-04-08,2017,665987.248566,639659.363097,738,21.770996
3494,Guana Tolomato Matanzas,Water Temperature,52,,,,,GTM,2017-04-01,2017-04-08,2017,659731.434296,654158.019057,743,22.544156


In [20]:
importlib.reload(idw_rk_x)

idw_rk_x.create_shp_season1(seasons_con_coord, shpCon_folder)

Number of data rows for BBS, DO_mgl, 1: 1
Shapefile for BBS: DO_mgl for period 1 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_1.shp
Number of data rows for BBS, DO_mgl, 2: 1
Shapefile for BBS: DO_mgl for period 2 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_2.shp
Number of data rows for BBS, DO_mgl, 3: 1
Shapefile for BBS: DO_mgl for period 3 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_3.shp
Number of data rows for BBS, DO_mgl, 4: 1
Shapefile for BBS: DO_mgl for period 4 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_4.shp
Number of data rows for BBS, DO_mgl, 5: 1
Shapefile for BBS: DO_mgl for period 5 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_5.shp
Number of data rows for BBS, DO_mgl, 6: 1
Shapefile for BBS: DO_mgl for period 6 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_6.shp
Number of data rows for BBS, DO_mgl, 7: 1
Shapefile for BBS: DO_mgl for period 7 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_7.shp
Number of data rows for BBS, DO_mg

Shapefile for BBS: Sal_ppt for period 9 of year 2021 has been saved as SHP_BBS_Sal_ppt_2021_9.shp
Number of data rows for BBS, Sal_ppt, 10: 1
Shapefile for BBS: Sal_ppt for period 10 of year 2021 has been saved as SHP_BBS_Sal_ppt_2021_10.shp
Number of data rows for BBS, Sal_ppt, 11: 1
Shapefile for BBS: Sal_ppt for period 11 of year 2021 has been saved as SHP_BBS_Sal_ppt_2021_11.shp
Number of data rows for BBS, Sal_ppt, 12: 1
Shapefile for BBS: Sal_ppt for period 12 of year 2021 has been saved as SHP_BBS_Sal_ppt_2021_12.shp
Number of data rows for BBS, Sal_ppt, 13: 1
Shapefile for BBS: Sal_ppt for period 13 of year 2021 has been saved as SHP_BBS_Sal_ppt_2021_13.shp
Number of data rows for BBS, Sal_ppt, 14: 1
Shapefile for BBS: Sal_ppt for period 14 of year 2021 has been saved as SHP_BBS_Sal_ppt_2021_14.shp
Number of data rows for BBS, Sal_ppt, 15: 1
Shapefile for BBS: Sal_ppt for period 15 of year 2021 has been saved as SHP_BBS_Sal_ppt_2021_15.shp
Number of data rows for BBS, Sal_ppt, 

Shapefile for BBS: Turb_ntu for period 16 of year 2021 has been saved as SHP_BBS_Turb_ntu_2021_16.shp
Number of data rows for BBS, Turb_ntu, 17: 1
Shapefile for BBS: Turb_ntu for period 17 of year 2021 has been saved as SHP_BBS_Turb_ntu_2021_17.shp
Number of data rows for BBS, Turb_ntu, 18: 1
Shapefile for BBS: Turb_ntu for period 18 of year 2021 has been saved as SHP_BBS_Turb_ntu_2021_18.shp
Number of data rows for BBS, Turb_ntu, 19: 1
Shapefile for BBS: Turb_ntu for period 19 of year 2021 has been saved as SHP_BBS_Turb_ntu_2021_19.shp
Number of data rows for BBS, Turb_ntu, 20: 1
Shapefile for BBS: Turb_ntu for period 20 of year 2021 has been saved as SHP_BBS_Turb_ntu_2021_20.shp
Number of data rows for BBS, Turb_ntu, 21: 1
Shapefile for BBS: Turb_ntu for period 21 of year 2021 has been saved as SHP_BBS_Turb_ntu_2021_21.shp
Number of data rows for BBS, Turb_ntu, 22: 1
Shapefile for BBS: Turb_ntu for period 22 of year 2021 has been saved as SHP_BBS_Turb_ntu_2021_22.shp
Number of data r

Shapefile for BBS: T_c for period 23 of year 2021 has been saved as SHP_BBS_T_c_2021_23.shp
Number of data rows for BBS, T_c, 24: 1
Shapefile for BBS: T_c for period 24 of year 2021 has been saved as SHP_BBS_T_c_2021_24.shp
Number of data rows for BBS, T_c, 25: 1
Shapefile for BBS: T_c for period 25 of year 2021 has been saved as SHP_BBS_T_c_2021_25.shp
Number of data rows for BBS, T_c, 26: 1
Shapefile for BBS: T_c for period 26 of year 2021 has been saved as SHP_BBS_T_c_2021_26.shp
Number of data rows for BBS, T_c, 27: 1
Shapefile for BBS: T_c for period 27 of year 2021 has been saved as SHP_BBS_T_c_2021_27.shp
Number of data rows for BBS, T_c, 28: 1
Shapefile for BBS: T_c for period 28 of year 2021 has been saved as SHP_BBS_T_c_2021_28.shp
Number of data rows for BBS, T_c, 29: 1
Shapefile for BBS: T_c for period 29 of year 2021 has been saved as SHP_BBS_T_c_2021_29.shp
Number of data rows for BBS, T_c, 30: 1
Shapefile for BBS: T_c for period 30 of year 2021 has been saved as SHP_BBS_

Shapefile for BB: DO_mgl for period 32 of year 2022 has been saved as SHP_BB_DO_mgl_2022_32.shp
Number of data rows for BB, DO_mgl, 33: 6
Shapefile for BB: DO_mgl for period 33 of year 2022 has been saved as SHP_BB_DO_mgl_2022_33.shp
Number of data rows for BB, DO_mgl, 34: 6
Shapefile for BB: DO_mgl for period 34 of year 2022 has been saved as SHP_BB_DO_mgl_2022_34.shp
Number of data rows for BB, DO_mgl, 35: 5
Shapefile for BB: DO_mgl for period 35 of year 2022 has been saved as SHP_BB_DO_mgl_2022_35.shp
Number of data rows for BB, DO_mgl, 36: 5
Shapefile for BB: DO_mgl for period 36 of year 2022 has been saved as SHP_BB_DO_mgl_2022_36.shp
Number of data rows for BB, DO_mgl, 37: 6
Shapefile for BB: DO_mgl for period 37 of year 2022 has been saved as SHP_BB_DO_mgl_2022_37.shp
Number of data rows for BB, DO_mgl, 38: 6
Shapefile for BB: DO_mgl for period 38 of year 2022 has been saved as SHP_BB_DO_mgl_2022_38.shp
Number of data rows for BB, DO_mgl, 39: 12
Shapefile for BB: DO_mgl for peri

...

Shapefile for EB: Turb_ntu for period 49 of year 2018 has been saved as SHP_EB_Turb_ntu_2018_49.shp
Number of data rows for EB, Turb_ntu, 50: 3
Shapefile for EB: Turb_ntu for period 50 of year 2018 has been saved as SHP_EB_Turb_ntu_2018_50.shp
Number of data rows for EB, Turb_ntu, 51: 3
Shapefile for EB: Turb_ntu for period 51 of year 2018 has been saved as SHP_EB_Turb_ntu_2018_51.shp
Number of data rows for EB, Turb_ntu, 52: 2
Shapefile for EB: Turb_ntu for period 52 of year 2018 has been saved as SHP_EB_Turb_ntu_2018_52.shp
Number of data rows for EB, T_c, 1: 3
Shapefile for EB: T_c for period 1 of year 2017 has been saved as SHP_EB_T_c_2017_1.shp
Number of data rows for EB, T_c, 2: 3
Shapefile for EB: T_c for period 2 of year 2017 has been saved as SHP_EB_T_c_2017_2.shp
Number of data rows for EB, T_c, 3: 3
Shapefile for EB: T_c for period 3 of year 2017 has been saved as SHP_EB_T_c_2017_3.shp
Number of data rows for EB, T_c, 4: 3
Shapefile for EB: T_c for period 4 of year 2017 has 

Shapefile for GTM: DO_mgl for period 8 of year 2016 has been saved as SHP_GTM_DO_mgl_2016_8.shp
Number of data rows for GTM, DO_mgl, 9: 4
Shapefile for GTM: DO_mgl for period 9 of year 2016 has been saved as SHP_GTM_DO_mgl_2016_9.shp
Number of data rows for GTM, DO_mgl, 10: 8
Shapefile for GTM: DO_mgl for period 10 of year 2016 has been saved as SHP_GTM_DO_mgl_2016_10.shp
Number of data rows for GTM, DO_mgl, 11: 4
Shapefile for GTM: DO_mgl for period 11 of year 2016 has been saved as SHP_GTM_DO_mgl_2016_11.shp
Number of data rows for GTM, DO_mgl, 12: 4
Shapefile for GTM: DO_mgl for period 12 of year 2016 has been saved as SHP_GTM_DO_mgl_2016_12.shp
Number of data rows for GTM, DO_mgl, 13: 4
Shapefile for GTM: DO_mgl for period 13 of year 2016 has been saved as SHP_GTM_DO_mgl_2016_13.shp
Number of data rows for GTM, DO_mgl, 14: 4
Shapefile for GTM: DO_mgl for period 14 of year 2016 has been saved as SHP_GTM_DO_mgl_2016_14.shp
Number of data rows for GTM, DO_mgl, 15: 3
Shapefile for GTM:

Shapefile for GTM: Sal_ppt for period 19 of year 2016 has been saved as SHP_GTM_Sal_ppt_2016_19.shp
Number of data rows for GTM, Sal_ppt, 20: 4
Shapefile for GTM: Sal_ppt for period 20 of year 2016 has been saved as SHP_GTM_Sal_ppt_2016_20.shp
Number of data rows for GTM, Sal_ppt, 21: 4
Shapefile for GTM: Sal_ppt for period 21 of year 2016 has been saved as SHP_GTM_Sal_ppt_2016_21.shp
Number of data rows for GTM, Sal_ppt, 22: 4
Shapefile for GTM: Sal_ppt for period 22 of year 2016 has been saved as SHP_GTM_Sal_ppt_2016_22.shp
Number of data rows for GTM, Sal_ppt, 23: 8
Shapefile for GTM: Sal_ppt for period 23 of year 2016 has been saved as SHP_GTM_Sal_ppt_2016_23.shp
Number of data rows for GTM, Sal_ppt, 24: 4
Shapefile for GTM: Sal_ppt for period 24 of year 2016 has been saved as SHP_GTM_Sal_ppt_2016_24.shp
Number of data rows for GTM, Sal_ppt, 25: 2
Shapefile for GTM: Sal_ppt for period 25 of year 2016 has been saved as SHP_GTM_Sal_ppt_2016_25.shp
Number of data rows for GTM, Sal_ppt

Shapefile for GTM: Turb_ntu for period 28 of year 2016 has been saved as SHP_GTM_Turb_ntu_2016_28.shp
Number of data rows for GTM, Turb_ntu, 29: 4
Shapefile for GTM: Turb_ntu for period 29 of year 2016 has been saved as SHP_GTM_Turb_ntu_2016_29.shp
Number of data rows for GTM, Turb_ntu, 30: 4
Shapefile for GTM: Turb_ntu for period 30 of year 2016 has been saved as SHP_GTM_Turb_ntu_2016_30.shp
Number of data rows for GTM, Turb_ntu, 31: 3
Shapefile for GTM: Turb_ntu for period 31 of year 2016 has been saved as SHP_GTM_Turb_ntu_2016_31.shp
Number of data rows for GTM, Turb_ntu, 32: 4
Shapefile for GTM: Turb_ntu for period 32 of year 2016 has been saved as SHP_GTM_Turb_ntu_2016_32.shp
Number of data rows for GTM, Turb_ntu, 33: 4
Shapefile for GTM: Turb_ntu for period 33 of year 2016 has been saved as SHP_GTM_Turb_ntu_2016_33.shp
Number of data rows for GTM, Turb_ntu, 34: 4
Shapefile for GTM: Turb_ntu for period 34 of year 2016 has been saved as SHP_GTM_Turb_ntu_2016_34.shp
Number of data r

Shapefile for GTM: T_c for period 48 of year 2017 has been saved as SHP_GTM_T_c_2017_48.shp
Number of data rows for GTM, T_c, 49: 4
Shapefile for GTM: T_c for period 49 of year 2017 has been saved as SHP_GTM_T_c_2017_49.shp
Number of data rows for GTM, T_c, 50: 4
Shapefile for GTM: T_c for period 50 of year 2017 has been saved as SHP_GTM_T_c_2017_50.shp
Number of data rows for GTM, T_c, 51: 4
Shapefile for GTM: T_c for period 51 of year 2017 has been saved as SHP_GTM_T_c_2017_51.shp
Number of data rows for GTM, T_c, 52: 4
Shapefile for GTM: T_c for period 52 of year 2017 has been saved as SHP_GTM_T_c_2017_52.shp


# 3. Cross Validation for IDW <a class="anchor" id="reg_cv_idw"></a>

In [21]:
# Empty the shapefile folder
idw_rk_x.delete_all_files(idwCon_folder)

In [22]:
df_week_table

,WaterBody,Parameter,Period,Filename,NumDataPoints,RMSE,ME,WbodyAcronym,startDate,endDate,Year
0,Big Bend Seagrasses,Dissolved Oxygen,1,,,,,BBS,2021-01-06,2021-01-12,2021
1,Big Bend Seagrasses,Dissolved Oxygen,2,,,,,BBS,2021-01-13,2021-01-19,2021
2,Big Bend Seagrasses,Dissolved Oxygen,3,,,,,BBS,2021-01-20,2021-01-26,2021
3,Big Bend Seagrasses,Dissolved Oxygen,4,,,,,BBS,2021-01-27,2021-02-02,2021
4,Big Bend Seagrasses,Dissolved Oxygen,5,,,,,BBS,2021-02-03,2021-02-09,2021
...,...,...,...,...,...,...,...,...,...,...,...
1035,Guana Tolomato Matanzas,Water Temperature,48,,,,,GTM,2017-03-04,2017-03-11,2017
1036,Guana Tolomato Matanzas,Water Temperature,49,,,,,GTM,2017-03-11,2017-03-18,2017
1037,Guana Tolomato Matanzas,Water Temperature,50,,,,,GTM,2017-03-18,2017-03-25,2017
1038,Guana Tolomato Matanzas,Water Temperature,51,,,,,GTM,2017-03-25,2017-04-01,2017


In [23]:
# If the number of data points is less than 3，skipping calculate IDW
idw_rk_x.idw_interpolation1(df_week_table, shpCon_folder, idwCon_folder, waterbody_extent, barrier_folder)

Processing file: SHP_BBS_DO_mgl_2021_1.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_1.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_2.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_2.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_3.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_3.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_4.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_4.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_5.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_5.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_6.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_6.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_7.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_7.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_8.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_8.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_9.s

Not enough data for IDW interpolation in SHP_BBS_Sal_ppt_2021_18.shp, skipping
Processing file: SHP_BBS_Sal_ppt_2021_19.shp
Not enough data for IDW interpolation in SHP_BBS_Sal_ppt_2021_19.shp, skipping
Processing file: SHP_BBS_Sal_ppt_2021_20.shp
Not enough data for IDW interpolation in SHP_BBS_Sal_ppt_2021_20.shp, skipping
Processing file: SHP_BBS_Sal_ppt_2021_21.shp
Not enough data for IDW interpolation in SHP_BBS_Sal_ppt_2021_21.shp, skipping
Shapefile not found for: SHP_BBS_Sal_ppt_2021_22.shp
Shapefile not found for: SHP_BBS_Sal_ppt_2021_23.shp
Shapefile not found for: SHP_BBS_Sal_ppt_2021_24.shp
Processing file: SHP_BBS_Sal_ppt_2021_25.shp
Not enough data for IDW interpolation in SHP_BBS_Sal_ppt_2021_25.shp, skipping
Processing file: SHP_BBS_Sal_ppt_2021_26.shp
Not enough data for IDW interpolation in SHP_BBS_Sal_ppt_2021_26.shp, skipping
Processing file: SHP_BBS_Sal_ppt_2021_27.shp
Not enough data for IDW interpolation in SHP_BBS_Sal_ppt_2021_27.shp, skipping
Processing file: S

Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_34.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021_35.shp
Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_35.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021_36.shp
Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_36.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021_37.shp
Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_37.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021_38.shp
Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_38.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021_39.shp
Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_39.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021_40.shp
Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_40.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021_41.shp
Not enough data for IDW interpolation in SHP_BBS_Turb_ntu_2021_41.shp, skipping
Processing file: SHP_BBS_Turb_ntu_2021

Not enough data for IDW interpolation in SHP_BBS_T_c_2021_52.shp, skipping
Processing file: SHP_BB_DO_mgl_2022_1.shp
File SHP_BB_DO_mgl_2022_1.shp has completed 4 cross-validation iterations.
Processing file: SHP_BB_DO_mgl_2022_2.shp
File SHP_BB_DO_mgl_2022_2.shp has completed 4 cross-validation iterations.
Processing file: SHP_BB_DO_mgl_2022_3.shp
File SHP_BB_DO_mgl_2022_3.shp has completed 5 cross-validation iterations.
Processing file: SHP_BB_DO_mgl_2022_4.shp
File SHP_BB_DO_mgl_2022_4.shp has completed 6 cross-validation iterations.
Processing file: SHP_BB_DO_mgl_2022_5.shp


KeyboardInterrupt: 

# 4. Monthly Results <a class="anchor" id="month"></a>

### Set the folder path for weekly result

In [29]:
shpCon_folder = gis_path + r"shapefiles/idw_month"
idwCon_folder = gis_path + r"raster_output/idw_month"

# shpCon_folder = gis_path + r"shapefiles_Con/month"
# idwCon_folder = gis_path + r"idw_Con/month"

In [30]:
area_ab = ["GTM","EB","CH","BB","BBS"]
period_type = [" 52 week"," Month"]
dfConTime["Period"] = ""
def select_data_period1(df,area,period):
    sheet_name = str(area) + str(period)
    df_period_table = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name)
    df_select_area = df[df["WbodyAcronym"]==str(area)]
    df_period_table['Start Date'] = pd.to_datetime(df_period_table['Start Date'])
    df_period_table['End Date']   = pd.to_datetime(df_period_table['End Date'])
    sub_dfs = []

    for index, row in df_period_table.iterrows():
        start_date = row['Start Date']
        end_date   = row['End Date']
        sub_df = df_select_area[(df_select_area['SampleDate'] >= start_date) & (df_select_area['SampleDate'] < end_date)]
        sub_df['Period'] = row["Month"]
        sub_dfs.append(sub_df)
        
    df_period = pd.concat(sub_dfs,ignore_index=True)
    return df_period

In [31]:
sel_month_temp = []
for each in area_ab:
    df_month_temp = select_data_period1(dfConTime,str(each)," Month")
    df_month_temp_group = df_month_temp.groupby(['WaterBody','ParameterName','ParameterUnits',
                                          'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym',"Period"])["ResultValue"].agg("mean").reset_index()
    sel_month_temp.append(df_month_temp_group)
df_month_select_Mean = pd.concat(sel_month_temp,ignore_index=True)

In [32]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
df_month_select_Mean[['x', 'y']] = df_month_select_Mean.apply(transform_coordinates, axis=1)

In [33]:
df_month_select_Mean

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,Period,ResultValue,x,y
0,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,6,5.142161,665054.340859,631868.366218
1,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,7,3.728834,665054.340859,631868.366218
2,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,8,6.632493,665054.340859,631868.366218
3,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.737041,-81.245953,GTM,6,5.606851,665987.248566,639659.363097
4,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.737041,-81.245953,GTM,7,6.354141,665987.248566,639659.363097
...,...,...,...,...,...,...,...,...,...,...,...,...
1057,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Summer,29.647203,-83.421196,BBS,9,29.325253,455952.242067,626846.365120
1058,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,1,13.373354,455952.242067,626846.365120
1059,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,2,17.077020,455952.242067,626846.365120
1060,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,12,19.022835,455952.242067,626846.365120


In [34]:
waterBody = ['Big Bend Seagrasses', 'Biscayne Bay', 'Charlotte Harbor', 'Estero Bay', 'Guana Tolomato Matanzas']
parameter = ['Dissolved Oxygen', 'Salinity', 'Turbidity', 'Water Temperature']
waterBody_list = []
parameter_list= []
month_list = []
for i in waterBody:
    for j in parameter:
        for k in range(1,13):
            waterBody_list.append(i)
            parameter_list.append(j)
            month_list.append(k)

In [35]:
df_month_table = pd.DataFrame({
    "WaterBody":waterBody_list,
    "Parameter":parameter_list,
    "Period":month_list
})
df_month_table["Filename"] = ""
df_month_table["NumDataPoints"] = ""
df_month_table["RMSE"] = ""
df_month_table["ME"] = ""
df_month_table["WbodyAcronym"] = df_month_table["WaterBody"].map(area_shortnames)

In [36]:
name_ab = df_month_table["WbodyAcronym"].unique()
para_list = df_month_table["Parameter"].unique()
dfs_sub = []
for each in name_ab:
    sheet_name_use = str(each) + " Month"
    df_period_table_used = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name_use)
    startDate = list(df_period_table_used["Start Date"])
    endDate   = list(df_period_table_used["End Date"])
    for para in para_list:
        df_temp_use = df_month_table[(df_month_table["WbodyAcronym"]==str(each))&(df_month_table["Parameter"]==str(para))]
        df_temp_use["startDate"] = startDate
        df_temp_use["endDate"]   = endDate
        dfs_sub.append(df_temp_use)
df_month_table = pd.concat(dfs_sub)
df_month_table["Year"] = df_month_table["startDate"].dt.year
df_month_table.to_csv(gis_path + 'month_table.csv')

In [37]:
idw_rk_x.fill_nan_rowids(df_month_select_Mean, 'RowID')

df_month_select_Mean['RowID'] = df_month_select_Mean['RowID'].astype(int)
df_month_select_Mean

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,Period,ResultValue,x,y,RowID
0,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,6,5.142161,665054.340859,631868.366218,1
1,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,7,3.728834,665054.340859,631868.366218,2
2,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.667071,-81.257403,GTM,8,6.632493,665054.340859,631868.366218,3
3,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.737041,-81.245953,GTM,6,5.606851,665987.248566,639659.363097,4
4,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,Fall,29.737041,-81.245953,GTM,7,6.354141,665987.248566,639659.363097,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Summer,29.647203,-83.421196,BBS,9,29.325253,455952.242067,626846.365120,1058
1058,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,1,13.373354,455952.242067,626846.365120,1059
1059,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,2,17.077020,455952.242067,626846.365120,1060
1060,Big Bend Seagrasses,Water Temperature,Degrees C,2021,Winter,29.647203,-83.421196,BBS,12,19.022835,455952.242067,626846.365120,1061


In [41]:
idw_rk_x.delete_all_files(shpCon_folder)

In [42]:
# Merge interested with latitude and longitude columns
seasons_con_coord = idw_rk_x.merge_with_lat_long1(df_month_table, df_month_select_Mean)
seasons_con_coord

,WaterBody,Parameter,Period,Filename,NumDataPoints,RMSE,ME,WbodyAcronym,startDate,endDate,Year,x,y,RowID,ResultValue
0,Big Bend Seagrasses,Dissolved Oxygen,1,,,,,BBS,2021-01-08,2021-02-06,2021,455952.242067,626846.365120,1012,8.356844
1,Big Bend Seagrasses,Dissolved Oxygen,2,,,,,BBS,2021-02-07,2021-03-08,2021,455952.242067,626846.365120,1004,7.433333
2,Big Bend Seagrasses,Dissolved Oxygen,2,,,,,BBS,2021-02-07,2021-03-08,2021,455952.242067,626846.365120,1013,7.730177
3,Big Bend Seagrasses,Dissolved Oxygen,3,,,,,BBS,2021-03-09,2021-04-07,2021,455952.242067,626846.365120,1005,7.490700
4,Big Bend Seagrasses,Dissolved Oxygen,4,,,,,BBS,2021-04-08,2021-05-07,2021,455952.242067,626846.365120,1006,6.859352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,Guana Tolomato Matanzas,Water Temperature,11,,,,,GTM,2017-02-05,2017-03-07,2017,653509.737698,674232.732990,234,18.666667
1062,Guana Tolomato Matanzas,Water Temperature,12,,,,,GTM,2017-03-07,2017-04-06,2017,665054.340859,631868.366218,229,21.517071
1063,Guana Tolomato Matanzas,Water Temperature,12,,,,,GTM,2017-03-07,2017-04-06,2017,665987.248566,639659.363097,231,20.061616
1064,Guana Tolomato Matanzas,Water Temperature,12,,,,,GTM,2017-03-07,2017-04-06,2017,659731.434296,654158.019057,233,20.107685


In [43]:
idw_rk_x.create_shp_season1(seasons_con_coord, shpCon_folder)

Number of data rows for BBS, DO_mgl, 1: 1
Shapefile for BBS: DO_mgl for period 1 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_1.shp
Number of data rows for BBS, DO_mgl, 2: 2
Shapefile for BBS: DO_mgl for period 2 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_2.shp
Number of data rows for BBS, DO_mgl, 3: 1
Shapefile for BBS: DO_mgl for period 3 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_3.shp
Number of data rows for BBS, DO_mgl, 4: 1
Shapefile for BBS: DO_mgl for period 4 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_4.shp
Number of data rows for BBS, DO_mgl, 5: 1
Shapefile for BBS: DO_mgl for period 5 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_5.shp
Number of data rows for BBS, DO_mgl, 6: 1
Shapefile for BBS: DO_mgl for period 6 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_6.shp
Number of data rows for BBS, DO_mgl, 7: 1
Shapefile for BBS: DO_mgl for period 7 of year 2021 has been saved as SHP_BBS_DO_mgl_2021_7.shp
Number of data rows for BBS, DO_mg

Shapefile for BB: DO_mgl for period 12 of year 2023 has been saved as SHP_BB_DO_mgl_2023_12.shp
Number of data rows for BB, Sal_ppt, 1: 6
Shapefile for BB: Sal_ppt for period 1 of year 2022 has been saved as SHP_BB_Sal_ppt_2022_1.shp
Number of data rows for BB, Sal_ppt, 2: 6
Shapefile for BB: Sal_ppt for period 2 of year 2022 has been saved as SHP_BB_Sal_ppt_2022_2.shp
Number of data rows for BB, Sal_ppt, 3: 6
Shapefile for BB: Sal_ppt for period 3 of year 2022 has been saved as SHP_BB_Sal_ppt_2022_3.shp
Number of data rows for BB, Sal_ppt, 4: 12
Shapefile for BB: Sal_ppt for period 4 of year 2022 has been saved as SHP_BB_Sal_ppt_2022_4.shp
Number of data rows for BB, Sal_ppt, 5: 5
Shapefile for BB: Sal_ppt for period 5 of year 2022 has been saved as SHP_BB_Sal_ppt_2022_5.shp
Number of data rows for BB, Sal_ppt, 6: 6
Shapefile for BB: Sal_ppt for period 6 of year 2022 has been saved as SHP_BB_Sal_ppt_2022_6.shp
Number of data rows for BB, Sal_ppt, 7: 12
Shapefile for BB: Sal_ppt for pe

Shapefile for CH: Turb_ntu for period 1 of year 2017 has been saved as SHP_CH_Turb_ntu_2017_1.shp
Number of data rows for CH, Turb_ntu, 2: 3
Shapefile for CH: Turb_ntu for period 2 of year 2017 has been saved as SHP_CH_Turb_ntu_2017_2.shp
Number of data rows for CH, Turb_ntu, 3: 3
Shapefile for CH: Turb_ntu for period 3 of year 2017 has been saved as SHP_CH_Turb_ntu_2017_3.shp
Number of data rows for CH, Turb_ntu, 4: 3
Shapefile for CH: Turb_ntu for period 4 of year 2017 has been saved as SHP_CH_Turb_ntu_2017_4.shp
Number of data rows for CH, Turb_ntu, 5: 3
Shapefile for CH: Turb_ntu for period 5 of year 2017 has been saved as SHP_CH_Turb_ntu_2017_5.shp
Number of data rows for CH, Turb_ntu, 6: 3
Shapefile for CH: Turb_ntu for period 6 of year 2017 has been saved as SHP_CH_Turb_ntu_2017_6.shp
Number of data rows for CH, Turb_ntu, 7: 5
Shapefile for CH: Turb_ntu for period 7 of year 2017 has been saved as SHP_CH_Turb_ntu_2017_7.shp
Number of data rows for CH, Turb_ntu, 8: 3
Shapefile for

Shapefile for EB: T_c for period 1 of year 2017 has been saved as SHP_EB_T_c_2017_1.shp
Number of data rows for EB, T_c, 2: 3
Shapefile for EB: T_c for period 2 of year 2017 has been saved as SHP_EB_T_c_2017_2.shp
Number of data rows for EB, T_c, 3: 3
Shapefile for EB: T_c for period 3 of year 2017 has been saved as SHP_EB_T_c_2017_3.shp
Number of data rows for EB, T_c, 4: 6
Shapefile for EB: T_c for period 4 of year 2017 has been saved as SHP_EB_T_c_2017_4.shp
Number of data rows for EB, T_c, 5: 3
Shapefile for EB: T_c for period 5 of year 2017 has been saved as SHP_EB_T_c_2017_5.shp
Number of data rows for EB, T_c, 6: 3
Shapefile for EB: T_c for period 6 of year 2017 has been saved as SHP_EB_T_c_2017_6.shp
Number of data rows for EB, T_c, 7: 6
Shapefile for EB: T_c for period 7 of year 2017 has been saved as SHP_EB_T_c_2017_7.shp
Number of data rows for EB, T_c, 8: 3
Shapefile for EB: T_c for period 8 of year 2017 has been saved as SHP_EB_T_c_2017_8.shp
Number of data rows for EB, T_

In [65]:
# Empty the shapefile folder
idw_rk_x.delete_all_files(idwCon_folder)

In [66]:
# If the number of data points is less than 3，skipping calculate IDW
idw_rk_x.idw_interpolation1(df_month_table, shpCon_folder, idwCon_folder, waterbody_extent, barrier_folder)

Processing file: SHP_BBS_DO_mgl_2021_1.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_1.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_2.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_2.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_3.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_3.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_4.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_4.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_5.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_5.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_6.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_6.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_7.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_7.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_8.shp
Not enough data for IDW interpolation in SHP_BBS_DO_mgl_2021_8.shp, skipping
Processing file: SHP_BBS_DO_mgl_2021_9.s

KeyboardInterrupt: 